In [1]:
import numpy as np
import pandas as pd

In [2]:
import time
import nltk
import re
import spacy
import gensim
import time
import pickle

C:\Users\Amit\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
assert gensim.models.doc2vec.FAST_VERSION > -1

In [4]:
df1  = pd.read_csv("~\Desktop\Final Projects\Intro to ML\\articles1.csv")
df2  = pd.read_csv("~\Desktop\Final Projects\Intro to ML\\articles2.csv")
df3  = pd.read_csv("~\Desktop\Final Projects\Intro to ML\\articles3.csv")

In [5]:
frames = [df1, df2, df3]
real = pd.concat(frames)

In [6]:
real.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [7]:
real = real[real['year'] == 2016]
real = real[(real['month'] == 10) | (real['month'] == 11)]

In [8]:
real.publication.unique()

array(['New York Times', 'Breitbart', 'CNN', 'Business Insider',
       'Atlantic', 'Fox News', 'Talking Points Memo', 'Buzzfeed News',
       'National Review', 'New York Post', 'Guardian', 'NPR', 'Reuters',
       'Vox', 'Washington Post'], dtype=object)

In [9]:
publication = ['New York Times', 'CNN', 'Business Insider',
               'Fox News', 'Buzzfeed News', 'New York Post',
               'Guardian', 'NPR', 'Reuters','Vox',
               'Washington Post']

In [10]:
real = real.loc[real['publication'].isin(publication)]

In [11]:
real.publication.unique()

array(['New York Times', 'CNN', 'Business Insider', 'Fox News',
       'Buzzfeed News', 'New York Post', 'Guardian', 'NPR', 'Reuters',
       'Vox', 'Washington Post'], dtype=object)

In [12]:
real = real.rename(index=str, columns={'content': 'text'})

In [13]:
real1 = real.filter(['text'])

In [14]:
real1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11521 entries, 2561 to 42059
Data columns (total 1 columns):
text    11521 non-null object
dtypes: object(1)
memory usage: 180.0+ KB


In [16]:
text = real1.text

In [20]:
for i in range(len(text)):
    if text[i] is np.NAN:
        text[i] = 'Dummy Text'

In [21]:
%%time
for i in range(len(text)):
    text[i] = text[i].lower()
    text[i] = re.sub("((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(com)(\S+)?)|((\S+)?(\@)(\S+)?)", "", text[i])
    text[i] = re.sub("[^A-Za-z0-9 ]", " ", text[i])

Wall time: 1min 17s


In [23]:
nlp = spacy.load('en_core_web_sm')

In [24]:
def lemmma(text):
    lem = []
    for doc in nlp.pipe(text):
        lem.append([token.lemma_ for token in doc if not (token.is_punct or token.is_space or token.is_stop or token.like_num)])      
    return lem

In [25]:
%%time
lem = lemmma(text)

Wall time: 34min 33s


In [26]:
%%time
phrases = Phrases(lem)
bigram = Phraser(phrases)
bigram_lem = list(bigram[lem])

Wall time: 44.1 s


In [27]:
%%time
phrases2 = Phrases(bigram_lem)
trigram = Phraser(phrases2)
trigram_lem = list(trigram[bigram_lem])

Wall time: 51.8 s


In [28]:
def lemma_combine(lis):
    parsed_articles = []
    
    for i in range(len(lis)):
        concat_art = ' '.join(lis[i])
        parsed_articles.append(concat_art)
    
    return parsed_articles

In [29]:
%%time
uni_lem_comb = lemma_combine(lem)
bi_lem_comb = lemma_combine(bigram_lem)
tri_lem_comb = lemma_combine(trigram_lem)

Wall time: 284 ms


In [30]:
token_uni = "real_token_unigram_text"
fileObject = open(token_uni,'wb') 
pickle.dump(lem,fileObject)   
fileObject.close()

In [31]:
token_bigrams = "real_token_bigram_text"
fileObject = open(token_bigrams,'wb') 
pickle.dump(lem,fileObject)   
fileObject.close()

In [32]:
token_trigrams = "real_token_trigram_text"
fileObject = open(token_trigrams,'wb') 
pickle.dump(trigram_lem,fileObject)   
fileObject.close()

In [33]:
unigrams = "real_unigram_text"
fileObject = open(unigrams,'wb') 
pickle.dump(uni_lem_comb,fileObject)   
fileObject.close()

In [34]:
bigrams = "real_bigram_text"
fileObject = open(bigrams,'wb') 
pickle.dump(bi_lem_comb,fileObject)   
fileObject.close()

In [35]:
trigrams = "real_trigram_text"
fileObject = open(trigrams,'wb')
pickle.dump(tri_lem_comb,fileObject)   
fileObject.close()